In [1]:
import json

import numpy as np

import pandas as pd

from IPython.display import display

In [2]:
# Configurazione opzioni Pandas

pd.set_option('display.max_columns', None)

In [3]:
# Definizione funzioni ausiliarie

def float_converter(s):
    s = s.replace(",", ".")
    return s

def percentage_calculator(row):
    if(row['espressi'] == 0):
        return 0
    else:
        return row['voti']/row['espressi']

In [4]:
# Filtro dataset per regioni

def filtra_dataset_per_regioni(file_path_dataset, sep=';', dtype={}):
    df_dataset = pd.read_csv(file_path_dataset, sep=sep, dtype=dtype)
    mask = (df_dataset['Nom Officiel Région'] == 'Hauts-de-France') | \
           (df_dataset['Nom Officiel Région'] == 'Grand Est') | \
           (df_dataset['Nom Officiel Région'] == 'Île-de-France') | \
           (df_dataset['Nom Officiel Région'] == 'Normandie') | \
           (df_dataset['Nom Officiel Région'] == 'Bretagne') | \
           (df_dataset['Nom Officiel Région'] == 'Bourgogne-Franche-Comté') | \
           (df_dataset['Nom Officiel Région'] == 'Centre-Val de Loire')
    df_dataset = df_dataset[mask]
    df_dataset.reset_index(drop=True, inplace=True)
    return df_dataset

In [5]:
# Leggo il file di configurazione
with open('config/config.json') as f:
    config_data = json.load(f)

# Ottengo i percorsi dei file
file_path_vaccinazioni_covid = config_data['path_covid_19_france_vaccinations_age_sexe_dep']
file_path_andamento_dei_prezzi_al_consumo = config_data['path_economicref_france_indice_prix_consommation']
file_path_risultati_delle_elezioni_legislative_1_turno = config_data['path_elections_france_legislatives_2022_1er_tour_par_bureau_de_vote']
file_path_risultati_delle_elezioni_legislative_2_turno = config_data['path_elections_france_legislatives_2022_2nd_tour_par_bureau_de_vote']
file_path_risultati_delle_elezioni_presidenziali_1_turno = config_data['path_elections_france_presidentielles_2022_1er_tour_par_bureau_de_vote']
file_path_risultati_delle_elezioni_presidenziali_2_turno = config_data['path_elections_france_presidentielles_2022_2nd_tour_par_bureau_de_vote']
file_path_dati_sulla_disoccupazione = config_data['path_labouref_france_departement_quarter_jobseeker']
file_path_andamento_dei_prezzi_dei_carburanti = config_data['path_prix_des_carburants_j_1_001']

file_path_risultati_elezioni_presidenziali_1_turno = config_data['path_risultati_elezioni_presidenziali_1_turno']
file_path_risultati_elezioni_presidenziali_2_turno = config_data['path_risultati_elezioni_presidenziali_2_turno']

In [6]:
# Caricamento e cleaning dataset vaccinazioni_covid

df_vaccinazioni_covid = filtra_dataset_per_regioni(file_path_vaccinazioni_covid)
df_vaccinazioni_covid.rename(columns={"Date": "data", "Nombre de doses n°1": "numero_prime_dosi", "Nombre cumulé de doses n°1":"numero_cumulativo_prime_dosi", "Type de variable":"tipo_variabile","Valeur de la variable":"valore_variabile","Libellé de la variable":"etichetta_variabile", "Nom Officiel Département":"nome_dipartimento", "Code Officiel Région":"codice_regione", "Nom Officiel Région":"nome_regione", "Code Officiel Département":"codice_dipartimento", "Nombre de vaccinations complètes (doses n°2)":"numero_seconde_dosi", "Nombre de personnes vaccinées avec une dose de rappel (3e dose)":"numero_terze_dosi", "Nombre de personnes vaccinées avec une deuxième dose de rappel (4e dose)":"numero_quarte_dosi", "Nombre de doses de rappel bivalent":"numero_dosi_bivalenti", "Nombre de personnes ayant reçu une troisième dose de rappel":"numero_persone_terza_dose", "Nombre cumulé de vaccinations complètes (doses n°2)":"numero_cumulativo_seconde dosi", "Nombre cumulé de personnes vaccinées avec une dose de rappel (3e dose)":"numero_cumulativo_terze_dosi", "Nombre cumulé de personnes vaccinées avec une deuxième dose de rappel (4e dose)":"numero_cumulativo_quarte_dosi", "Nombre cumulé de personnes vaccinées avec une dose de rappel bivalent":"numero_cumulativo_dosi_bivalenti", "Nombre cumulé de personnes ayant reçu une troisième dose de rappel":"numero_cumulativo_persone_terza_dose_richiamo", "Couverture vaccinale partielle (dose n°1)":"copertura_vaccinale_prima_dose", "Couverture vaccinale complète (dose n°2)":"copertura_vaccinale_seconda_dose", "Couverture vaccinale rappel/3e dose":"copertura_vaccinale_terza_dose", "Couverture vaccinale 2e rappel/4e dose":"copertura_vaccinale_quarta_dose", "Couverture vaccinale du rappel bivalent":"copertura_vaccinale_dose_bivalente", "Couverture vaccinale de la troisième dose de rappel":"copertura_vaccinale_terza_dose_richiamo"}, inplace=True)
df_vaccinazioni_covid.drop(['numero_persone_terza_dose','numero_cumulativo_persone_terza_dose_richiamo', 'copertura_vaccinale_terza_dose_richiamo'], axis=1, inplace=True)
new_ordered_columns = ["data","nome_regione","codice_regione","nome_dipartimento","codice_dipartimento","tipo_variabile","etichetta_variabile","valore_variabile","numero_prime_dosi","numero_seconde_dosi","numero_terze_dosi","numero_quarte_dosi","numero_cumulativo_prime_dosi","numero_cumulativo_seconde dosi","numero_cumulativo_terze_dosi","numero_cumulativo_quarte_dosi","numero_dosi_bivalenti","numero_cumulativo_dosi_bivalenti", "copertura_vaccinale_prima_dose", "copertura_vaccinale_seconda_dose", "copertura_vaccinale_terza_dose", "copertura_vaccinale_quarta_dose", "copertura_vaccinale_dose_bivalente"]
df_vaccinazioni_covid = df_vaccinazioni_covid.reindex(columns=new_ordered_columns)
#df_vaccinazioni_covid.head(10)

In [ ]:
# Caricamento e cleaning dataset andamento_prezzi_consumo

df_andamento_prezzi_consumo = pd.read_csv(file_path_andamento_dei_prezzi_al_consumo, sep=';')
df_andamento_prezzi_consumo.rename(columns={"Période" : "periodo", "Indice" : "suggerimento", "Valeur": "valore","Code" : "codice","Type" : "tipo","Sous-type" : "sottotipo", "Base" : "base", "Population" : "popolazione", "Zone concernée" : "zona_interessata", "Code COICOP" : "codice_COICOP", "Désignation COICOP" : "designazione_COICOP", "Série arrêtée" : "serie_interrotta", "Référence INSEE" : "riferimento_INSEE"}, inplace=True)
#df_andamento_prezzi_consumo.head(10)

In [ ]:
# Caricamento e cleaning dataset risultati_elezioni_legislative_1_turno

df_risultati_elezioni_legislative_1_turno = filtra_dataset_per_regioni(file_path_risultati_delle_elezioni_legislative_1_turno, dtype={"Code du département": str, "Code de la commune": str, "Code du b.vote": str})
df_risultati_elezioni_legislative_1_turno.rename(columns={"Code du département" : "codice_dipartimento", "Libellé du département" : "nome_dipartimento", "Code de la circonscription" : "codice_della_circoscrizione", "Libellé de la circonscription" : "nome_della_circoscrizione", "Code de la commune" : "codice_del_comune", "Libellé de la commune" : "nome_del_comune", "Code du b.vote" : "codice_del_seggio_elettorale", "Inscrits" : "iscritti", "Abstentions" : "astenuti", "% Abs/Ins" : "% astenuti/iscritti", "Votants" : "votanti", "% Vot/Ins" : "% votanti/iscritti", "Blancs" : "bianche", "% Blancs/Ins" : "% bianche/iscritti", "% Blancs/Vot" : "% bianche/votanti", "Nuls" : "nulle", "% Nuls/Ins" : "% nulle/iscritti", "% Nuls/Vot" : "% nulle/votanti", "Exprimés" : "espressi", "% Exp/Ins" : "% espressi/iscritti", "% Exp/Vot" : "% espressi/votanti", "N°Panneau" : "pannello_numero", "Sexe" : "genere", "Nom" : "cognome", "Prénom" : "nome", "Nuance" : "partito", "Voix" : "voti", "% Voix/Ins" : "% voti/iscritti", "% Voix/Exp" : "% voti/iscritti", "Code Officiel EPCI" : "codice_ufficiale_EPCI", "Nom Officiel EPCI" : "nome_ufficiale_dello_EPCI", "Code Officiel Région" : "codice_ufficiale_regionale", "Nom Officiel Région" : "nome_ufficiale_regionale", "location" : "coordinate", "lib_du_b_vote" : "etichetta_del_seggio_elettorale", "scrutin_code" : "codice_scrutinio"}, inplace=True)
#df_risultati_elezioni_legislative_1_turno.head(10)

In [ ]:
# Caricamento e cleaning dataset risultati_elezioni_legislative_2_turno

df_risultati_elezioni_legislative_2_turno = filtra_dataset_per_regioni(file_path_risultati_delle_elezioni_legislative_2_turno, dtype={"Code du département": str, "Code de la commune": str, "Code du b.vote": str})
df_risultati_elezioni_legislative_2_turno.rename(columns={"Code du département" : "codice_dipartimento", "Libellé du département" : "nome_dipartimento", "Code de la circonscription" : "codice_della_circoscrizione", "Libellé de la circonscription" : "nome_della_circoscrizione", "Code de la commune" : "codice_del_comune", "Libellé de la commune" : "nome_del_comune", "Code du b.vote" : "codice_del_seggio_elettorale", "Inscrits" : "iscritti", "Abstentions" : "astenuti", "% Abs/Ins" : "% astenuti/iscritti", "Votants" : "votanti", "% Vot/Ins" : "% votanti/iscritti", "Blancs" : "bianche", "% Blancs/Ins" : "% bianche/iscritti", "% Blancs/Vot" : "% bianche/votanti", "Nuls" : "nulle", "% Nuls/Ins" : "% nulle/iscritti", "% Nuls/Vot" : "% nulle/votanti", "Exprimés" : "espressi", "% Exp/Ins" : "% espressi/iscritti", "% Exp/Vot" : "% espressi/votanti", "N°Panneau" : "pannello_numero", "Sexe" : "genere", "Nom" : "cognome", "Prénom" : "nome", "Nuance" : "partito", "Voix" : "voti", "% Voix/Ins" : "% voti/iscritti", "% Voix/Exp" : "% voti/iscritti", "Code Officiel EPCI" : "codice_ufficiale_EPCI", "Nom Officiel EPCI" : "nome_ufficiale_dello_EPCI", "Code Officiel Région" : "codice_ufficiale_regionale", "Nom Officiel Région" : "nome_ufficiale_regionale", "location" : "coordinate", "lib_du_b_vote" : "etichetta_del_seggio_elettorale", "scrutin_code" : "codice_scrutinio"}, inplace=True)
#df_risultati_elezioni_legislative_2_turno.head(10)

In [ ]:
# Caricamento e cleaning dataset risultati_elezioni_presidenziali_1_turno

df_risultati_elezioni_presidenziali_1_turno = filtra_dataset_per_regioni(file_path_risultati_delle_elezioni_presidenziali_1_turno, dtype={"Code du département": str, "Code de la commune": str, "Code du b.vote": str})
df_risultati_elezioni_presidenziali_1_turno.rename(columns={"Code du département" : "codice_dipartimento", "Libellé du département" : "nome_dipartimento", "Code de la circonscription" : "codice_circoscrizione", "Libellé de la circonscription" : "nome_circoscrizione", "Code de la commune" : "codice_comune", "Libellé de la commune" : "nome_comune", "Code du b.vote" : "codice_seggio_elettorale", "Inscrits" : "iscritti", "Abstentions" : "astenuti", "% Abs/Ins" : "% astenuti/iscritti", "Votants" : "votanti", "% Vot/Ins" : "% votanti/iscritti", "Blancs" : "bianche", "% Blancs/Ins" : "% bianche/iscritti", "% Blancs/Vot" : "% bianche/votanti", "Nuls" : "nulle", "% Nuls/Ins" : "% nulle/iscritti", "% Nuls/Vot" : "% nulle/votanti", "Exprimés" : "espressi", "% Exp/Ins" : "% espressi/iscritti", "% Exp/Vot" : "% espressi/votanti", "N°Panneau" : "pannello_numero", "Sexe" : "genere", "Nom" : "cognome", "Prénom" : "nome", "Nuance" : "partito", "Voix" : "voti", "% Voix/Ins" : "% voti/iscritti", "% Voix/Exp" : "% voti/espressi", "Code Officiel EPCI" : "codice_EPCI", "Nom Officiel EPCI" : "nome_EPCI", "Code Officiel Région" : "codice_regione", "Nom Officiel Région" : "nome_regione", "location" : "coordinate", "lib_du_b_vote" : "etichetta_seggio_elettorale", "scrutin_code" : "codice_scrutinio"}, inplace=True)
new_ordered_columns = ["codice_regione","nome_regione","codice_dipartimento","nome_dipartimento","codice_EPCI", "nome_EPCI","codice_comune","nome_comune","codice_circoscrizione","nome_circoscrizione","codice_seggio_elettorale","iscritti","astenuti","% astenuti/iscritti","votanti","% votanti/iscritti","bianche","% bianche/iscritti","% bianche/votanti","nulle","% nulle/iscritti","% nulle/votanti","espressi","% espressi/iscritti","% espressi/votanti","pannello_numero","genere","cognome","nome","partito","voti","% voti/iscritti","% voti/espressi","coordinate","etichetta_seggio_elettorale","codice_scrutinio"]
df_risultati_elezioni_presidenziali_1_turno = df_risultati_elezioni_presidenziali_1_turno.reindex(columns=new_ordered_columns)
df_risultati_elezioni_presidenziali_1_turno['% voti/espressi'] = df_risultati_elezioni_presidenziali_1_turno['% voti/espressi'].map(float_converter)
df_risultati_elezioni_presidenziali_1_turno = df_risultati_elezioni_presidenziali_1_turno.astype({"codice_regione": "Int64", "codice_EPCI": "Int64", "codice_circoscrizione": "Int64", "% voti/espressi": np.float64}, errors='ignore')
df_risultati_elezioni_presidenziali_1_turno = df_risultati_elezioni_presidenziali_1_turno.astype({"codice_regione": str, "codice_EPCI": str, "codice_circoscrizione": str})
df_risultati_elezioni_presidenziali_1_turno.to_csv(file_path_risultati_elezioni_presidenziali_1_turno, index=False)
df_risultati_elezioni_presidenziali_1_turno.head(10)

In [ ]:
# Caricamento e cleaning dataset risultati_elezioni_presidenziali_2_turno

df_risultati_elezioni_presidenziali_2_turno = filtra_dataset_per_regioni(file_path_risultati_delle_elezioni_presidenziali_2_turno, dtype={"Code du département": str, "Code de la commune": str, "Code du b.vote": str})
df_risultati_elezioni_presidenziali_2_turno.rename(columns={"Code du département" : "codice_dipartimento", "Libellé du département" : "nome_dipartimento", "Code de la circonscription" : "codice_circoscrizione", "Libellé de la circonscription" : "nome_circoscrizione", "Code de la commune" : "codice_comune", "Libellé de la commune" : "nome_comune", "Code du b.vote" : "codice_seggio_elettorale", "Inscrits" : "iscritti", "Abstentions" : "astenuti", "% Abs/Ins" : "% astenuti/iscritti", "Votants" : "votanti", "% Vot/Ins" : "% votanti/iscritti", "Blancs" : "bianche", "% Blancs/Ins" : "% bianche/iscritti", "% Blancs/Vot" : "% bianche/votanti", "Nuls" : "nulle", "% Nuls/Ins" : "% nulle/iscritti", "% Nuls/Vot" : "% nulle/votanti", "Exprimés" : "espressi", "% Exp/Ins" : "% espressi/iscritti", "% Exp/Vot" : "% espressi/votanti", "N°Panneau" : "pannello_numero", "Sexe" : "genere", "Nom" : "cognome", "Prénom" : "nome", "Nuance" : "partito", "Voix" : "voti", "% Voix/Ins" : "% voti/iscritti", "% Voix/Exp" : "% voti/espressi", "Code Officiel EPCI" : "codice_EPCI", "Nom Officiel EPCI" : "nome_EPCI", "Code Officiel Région" : "codice_regione", "Nom Officiel Région" : "nome_regione", "location" : "coordinate", "lib_du_b_vote" : "etichetta_seggio_elettorale", "scrutin_code" : "codice_scrutinio"}, inplace=True)
new_ordered_columns = ["codice_regione","nome_regione","codice_dipartimento","nome_dipartimento","codice_EPCI", "nome_EPCI","codice_comune","nome_comune","codice_circoscrizione","nome_circoscrizione","codice_seggio_elettorale","iscritti","astenuti","% astenuti/iscritti","votanti","% votanti/iscritti","bianche","% bianche/iscritti","% bianche/votanti","nulle","% nulle/iscritti","% nulle/votanti","espressi","% espressi/iscritti","% espressi/votanti","pannello_numero","genere","cognome","nome","partito","voti","% voti/iscritti","% voti/espressi","coordinate","etichetta_seggio_elettorale","codice_scrutinio"]
df_risultati_elezioni_presidenziali_2_turno = df_risultati_elezioni_presidenziali_2_turno.reindex(columns=new_ordered_columns)
df_risultati_elezioni_presidenziali_2_turno['% voti/espressi'] = df_risultati_elezioni_presidenziali_2_turno['% voti/espressi'].map(float_converter)
df_risultati_elezioni_presidenziali_2_turno = df_risultati_elezioni_presidenziali_2_turno.astype({"codice_regione": "Int64", "codice_EPCI": "Int64", "codice_circoscrizione": "Int64", "% voti/espressi": np.float64}, errors='ignore')
df_risultati_elezioni_presidenziali_2_turno = df_risultati_elezioni_presidenziali_2_turno.astype({"codice_regione": str, "codice_EPCI": str, "codice_circoscrizione": str})
df_risultati_elezioni_presidenziali_2_turno.to_csv(file_path_risultati_elezioni_presidenziali_2_turno, index=False)
#df_risultati_elezioni_presidenziali_2_turno.head(10)

In [8]:
# Caricamento e cleaning dataset disoccupazione

df_disoccupazione = filtra_dataset_per_regioni(file_path_dati_sulla_disoccupazione)
df_disoccupazione.rename(columns={"Période (Trimestre)": "periodo_trimestre","Date":"data","Catégorie":"categoria",
                                  "Tranche d'âge":"fascia_eta","Nb moyen demandeur emploi":"numero_medio_persone_cercanti_lavoro",
                                  "Nb moyen demandeur emploi Homme":"numero_medio_uomini_cercanti_lavoro","Nb moyen demandeur emploi Femme":"numero_medio_donne_cercanti_lavoro",
                                  "Différence Nb demandeur emploi trimestre précédent":"differenza_con_cercanti_lavoro_trimestre_prec",
                                  "Nb moyen entrées":"numero_medio_entrate","Nb moyen sorties":"numero_medio_uscite",
                                  "Nb moyen demandeur emploi longue durée":"numero_medio_persone_cercanti_lavoro_lungo_termine",
                                  "Code Officiel Département":"codice_dipartimento","Code Officiel Région":"codice_regione",
                                  "Nom Officiel Région":"nome_regione","Code Iso 3166-3 Zone":"Code Iso 3166-3 Zona",
                                  "Nom Officiel Département":"nome_dipartimento"}, inplace=True)
df_disoccupazione.drop(["periodo_trimestre", "categoria", "fascia_eta", "numero_medio_uomini_cercanti_lavoro", "numero_medio_donne_cercanti_lavoro",
                        "numero_medio_entrate", "numero_medio_uscite", "numero_medio_persone_cercanti_lavoro_lungo_termine", "Code Iso 3166-3 Zona"],
                        axis=1, inplace=True)
new_ordered_columns = ["data","nome_regione","codice_regione","nome_dipartimento","codice_dipartimento","numero_medio_persone_cercanti_lavoro","differenza_con_cercanti_lavoro_trimestre_prec"]
df_disoccupazione = df_disoccupazione.reindex(columns=new_ordered_columns)
#df_disoccupazione.head(10)

In [ ]:
# Caricamento e cleaning dataset andamento_prezzi_carburanti

df_andamento_prezzi_carburanti = filtra_dataset_per_regioni(file_path_andamento_dei_prezzi_dei_carburanti, dtype={"Code officiel commune ou arrondissement": str, "Nom Officiel Département": str, "Code Officiel Département": str})
df_andamento_prezzi_carburanti.rename(columns={"Timetable":"orario","Identifiant":"identificativo","Code Postal":"cap","Présence":"presenza","Adresse":"indirizzo","Nom Officiel Commune / Arrondissement Municipal Majuscule":"nome_comune","Code officiel commune ou arrondissement":"codice_comune","Nom Officiel Département":"nome_dipartimento","Code Officiel Département":"codice_dipartimento","Nom Officiel Région":"nome_regione","Code Officiel Région":"codice_regione","Nom Officiel EPCI":"nome_EPCI","Code Officiel EPCI":"codice_EPCI","Automate 24-24":"automatico","Carburant":"carburante","Rupture":"interruzione","Mise à jour":"aggiornamento","Prix Gazole":"prezzo_diesel","Prix SP95":"prezzo_SP95","Prix SP98":"prezzo_SP98","Prix GPLc":"prezzo_GPLc","Prix E10":"prezzo_E10","Prix E85":"prezzo_E85","Services":"servizi","Marque":"marca","Nom":"nome","Geo Point":"punto_geografico"}, inplace=True)
new_ordered_columns = ["orario","identificativo","cap","presenza","indirizzo","nome_regione","codice_regione","nome_dipartimento","codice_dipartimento","nome_EPCI","codice_EPCI","nome_comune","codice_comune","automatico","carburante","interruzione","aggiornamento","prezzo_diesel","prezzo_SP95","prezzo_SP98","prezzo_GPLc","prezzo_E10","prezzo_E85","servizi","marca","nome","punto_geografico"]
df_andamento_prezzi_carburanti = df_andamento_prezzi_carburanti.reindex(columns=new_ordered_columns)
#df_andamento_prezzi_carburanti.head(10)

In [ ]:
# Query n.1

# • I 20 dipartimenti con la maggiore percentuale di voti per Macron
#     • Primo e secondo turno

# Raggruppa i dati sulle colonne 'nome_dipartimento' e 'cognome' e ottiene la somma dei valori della colonna 'voti' e della colonna 'espressi'
df_final = df_risultati_elezioni_presidenziali_1_turno.groupby(['nome_dipartimento', 'cognome'], as_index=False)[['voti','espressi']].sum()
df_final = df_final[df_final['cognome']=='MACRON']
df_final['% voti_a_favore/voti_totali'] = df_final.apply(percentage_calculator, axis=1)
df_final = df_final.sort_values(by="% voti_a_favore/voti_totali",ascending=False)
df_final.reset_index(drop=True, inplace=True)
df_query_1_turno = df_final[:20]
display(df_query_1_turno)
lista_dipartimenti_1_turno = df_query_1_turno["nome_dipartimento"].values.tolist()
display(lista_dipartimenti_1_turno)

# Raggruppa i dati sulle colonne 'nome_dipartimento' e 'cognome' e ottiene la somma dei valori della colonna 'voti' e della colonna 'espressi'
df_final = df_risultati_elezioni_presidenziali_2_turno.groupby(['nome_dipartimento', 'cognome'], as_index=False)[['voti','espressi']].sum()
df_final = df_final[df_final['cognome']=='MACRON']
df_final['% voti_a_favore/voti_totali'] = df_final.apply(percentage_calculator, axis=1)
df_final = df_final.sort_values(by="% voti_a_favore/voti_totali",ascending=False)
df_final.reset_index(drop=True, inplace=True)
df_query_2_turno = df_final[:20]
display(df_query_2_turno)
lista_dipartimenti_2_turno = df_query_2_turno["nome_dipartimento"].values.tolist()
display(lista_dipartimenti_2_turno)

In [ ]:
# Query n.2

# I 20 dipartimenti con la minore percentuale di voti per Macron
#     • Primo e secondo turno

# Raggruppa i dati sulle colonne 'nome_dipartimento' e 'cognome' e ottiene la somma dei valori della colonna 'voti' e della colonna 'espressi'
df_final = df_risultati_elezioni_presidenziali_1_turno.groupby(['nome_dipartimento', 'cognome'], as_index=False)[['voti','espressi']].sum()
df_final = df_final[df_final['cognome']=='MACRON']
df_final['% voti_a_favore/voti_totali'] = df_final.apply(percentage_calculator, axis=1)
df_final = df_final.sort_values(by="% voti_a_favore/voti_totali",ascending=False)
df_final.reset_index(drop=True, inplace=True)
df_query_1_turno = df_final[-20:]
display(df_query_1_turno)
lista_dipartimenti_1_turno = df_query_1_turno["nome_dipartimento"].values.tolist()
display(lista_dipartimenti_1_turno)

# Raggruppa i dati sulle colonne 'nome_dipartimento' e 'cognome' e ottiene la somma dei valori della colonna 'voti' e della colonna 'espressi'
df_final = df_risultati_elezioni_presidenziali_2_turno.groupby(['nome_dipartimento', 'cognome'], as_index=False)[['voti','espressi']].sum()
df_final = df_final[df_final['cognome']=='MACRON']
df_final['% voti_a_favore/voti_totali'] = df_final.apply(percentage_calculator, axis=1)
df_final = df_final.sort_values(by="% voti_a_favore/voti_totali",ascending=False)
df_final.reset_index(drop=True, inplace=True)
df_query_2_turno = df_final[-20:]
display(df_query_2_turno)
lista_dipartimenti_2_turno = df_query_2_turno["nome_dipartimento"].values.tolist()
display(lista_dipartimenti_2_turno)

In [ ]:
# Query n.3

# I 20 dipartimenti con la maggiore percentuale di voti per Marine Le Pen
#    • Primo e secondo turno

# Raggruppa i dati sulle colonne 'nome_dipartimento' e 'cognome' e ottiene la somma dei valori della colonna 'voti' e della colonna 'espressi'
df_final = df_risultati_elezioni_presidenziali_1_turno.groupby(['nome_dipartimento', 'cognome'], as_index=False)[['voti','espressi']].sum()
df_final = df_final[df_final['cognome']=='LE PEN']
df_final['% voti_a_favore/voti_totali'] = df_final.apply(percentage_calculator, axis=1)
df_final = df_final.sort_values(by="% voti_a_favore/voti_totali",ascending=False)
df_final.reset_index(drop=True, inplace=True)
df_query_1_turno = df_final[:20]
display(df_query_1_turno)
lista_dipartimenti_1_turno = df_query_1_turno["nome_dipartimento"].values.tolist()
display(lista_dipartimenti_1_turno)

# Raggruppa i dati sulle colonne 'nome_dipartimento' e 'cognome' e ottiene la somma dei valori della colonna 'voti' e della colonna 'espressi'
df_final = df_risultati_elezioni_presidenziali_2_turno.groupby(['nome_dipartimento', 'cognome'], as_index=False)[['voti','espressi']].sum()
df_final = df_final[df_final['cognome']=='LE PEN']
df_final['% voti_a_favore/voti_totali'] = df_final.apply(percentage_calculator, axis=1)
df_final = df_final.sort_values(by="% voti_a_favore/voti_totali",ascending=False)
df_final.reset_index(drop=True, inplace=True)
df_query_2_turno = df_final[:20]
display(df_query_2_turno)
lista_dipartimenti_2_turno = df_query_2_turno["nome_dipartimento"].values.tolist()
display(lista_dipartimenti_2_turno)

In [ ]:
# Query n.4

# I 20 dipartimenti con la minore percentuale di voti per Marine Le Pen
#     • Primo e secondo turno

# Raggruppa i dati sulle colonne 'nome_dipartimento' e 'cognome' e ottiene la somma dei valori della colonna 'voti' e della colonna 'espressi'
df_final = df_risultati_elezioni_presidenziali_1_turno.groupby(['nome_dipartimento', 'cognome'], as_index=False)[['voti','espressi']].sum()
df_final = df_final[df_final['cognome']=='LE PEN']
df_final['% voti_a_favore/voti_totali'] = df_final.apply(percentage_calculator, axis=1)
df_final = df_final.sort_values(by="% voti_a_favore/voti_totali",ascending=False)
df_final.reset_index(drop=True, inplace=True)
df_query_1_turno = df_final[-20:]
display(df_query_1_turno)
lista_dipartimenti_1_turno = df_query_1_turno["nome_dipartimento"].values.tolist()
display(lista_dipartimenti_1_turno)

# Raggruppa i dati sulle colonne 'nome_dipartimento' e 'cognome' e ottiene la somma dei valori della colonna 'voti' e della colonna 'espressi'
df_final = df_risultati_elezioni_presidenziali_2_turno.groupby(['nome_dipartimento', 'cognome'], as_index=False)[['voti','espressi']].sum()
df_final = df_final[df_final['cognome']=='LE PEN']
df_final['% voti_a_favore/voti_totali'] = df_final.apply(percentage_calculator, axis=1)
df_final = df_final.sort_values(by="% voti_a_favore/voti_totali",ascending=False)
df_final.reset_index(drop=True, inplace=True)
df_query_2_turno = df_final[-20:]
display(df_query_2_turno)
lista_dipartimenti_2_turno = df_query_2_turno["nome_dipartimento"].values.tolist()
display(lista_dipartimenti_2_turno)

In [ ]:
# Query n.5

In [13]:
# Query n.6 e n.7

#seleziono il primo trimestre del 2022 e sommo il numero di persone in cerca di lavoro
primo_trimestre = df_disoccupazione[(pd.to_datetime(df_disoccupazione['data']).dt.year == 2022 ) & (pd.to_datetime(df_disoccupazione['data']).dt.quarter == 1)]
primo_trimestre_sum = primo_trimestre.groupby('nome_dipartimento')['numero_medio_persone_cercanti_lavoro'].sum()

#seleziono i restanti trimestri del 2022 e sommo la differenza delle persone in cerca di lavoro rispetto al trimestre precedente
trimestri_successivi = df_disoccupazione[(pd.to_datetime(df_disoccupazione['data']).dt.year == 2022 ) & (pd.to_datetime(df_disoccupazione['data']).dt.quarter > 1)]
trimestri_successivi_sum = trimestri_successivi.groupby('nome_dipartimento')['differenza_con_cercanti_lavoro_trimestre_prec'].sum()

#unisco i dati e sommo il numero di disoccupati al primo trimeste con la somma delle differenze
risultato = pd.merge(primo_trimestre_sum, trimestri_successivi_sum, on='nome_dipartimento')
risultato['disoccupazione_totale'] = risultato['numero_medio_persone_cercanti_lavoro'] + risultato['differenza_con_cercanti_lavoro_trimestre_prec']

#seleziono i 20 dipartimenti con la maggiore e minore disoccupazione totale
risultato_top_20 = risultato.nlargest(20, 'disoccupazione_totale')
risultato_bottom_20 = risultato.nsmallest(20, 'disoccupazione_totale')

display("I 20 dipartimenti con la maggiore disoccupazione", risultato_top_20)
display("I 20 dipartimenti con la minore disoccupazione", risultato_bottom_20)

'I 20 dipartimenti con la maggiore disoccupazione'

,numero_medio_persone_cercanti_lavoro,differenza_con_cercanti_lavoro_trimestre_prec,disoccupazione_totale
nome_dipartimento,,,
Nord,757050,-14890.0,742160.0
Paris,592520,-12630.0,579890.0
Seine-Saint-Denis,520890,-9930.0,510960.0
Pas-de-Calais,380650,-14020.0,366630.0
Hauts-de-Seine,353720,-8310.0,345410.0
Val-de-Marne,332940,-7710.0,325230.0
Seine-et-Marne,326900,-4370.0,322530.0
Seine-Maritime,332370,-16490.0,315880.0
Val-d'Oise,318650,-8030.0,310620.0


'I 20 dipartimenti con la minore disoccupazione'

,numero_medio_persone_cercanti_lavoro,differenza_con_cercanti_lavoro_trimestre_prec,disoccupazione_totale
nome_dipartimento,,,
Territoire de Belfort,38070,-1500.0,36570.0
Haute-Marne,37680,-1030.0,36650.0
Nièvre,39420,-440.0,38980.0
Meuse,41370,-940.0,40430.0
Indre,50570,-1900.0,48670.0
Haute-Saône,52530,-1750.0,50780.0
Jura,52490,-480.0,52010.0
Orne,59910,-2110.0,57800.0
Cher,72480,-2440.0,70040.0


In [ ]:
# Query n.8

In [ ]:
# Query n.9

In [ ]:
# Query n.10

In [ ]:
# Query n.11

In [ ]:
# Query n.12

In [ ]:
# Query n.13

In [ ]:
# Query n.14